In [87]:
!pip install pymysql
!pip install apyori
!pip install pandas
!pip install matplotlib
!pip install seaborn

In [88]:
# Imported Libraries
import pandas as pd
import pymysql

# Data visualisation tools for later.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori

#######Connection to Client Database
host="qut-ds.c2m09j1oykve.ap-southeast-2.rds.amazonaws.com"
port=4005
dbname="qut_ds1"
user="qut_ds1"
password="GaAVSqC#9JR8"
conn = pymysql.connect(host=host, user=user,port=port,password=password, db=dbname)


print(conn)

In [89]:
####Checking queries

###Customer Personal details
##  add the desired rows in the query from the three tables
df_customer_details = pd.read_sql("select s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member h  on s.u_ndis_number = h.membership_number left join libe_leapinprod_memberregion r on r.MemberId =  s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null;", con=conn) 
print(df_customer_details.info())
print(df_customer_details)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   u_disabilities   6241 non-null   object 
 1   u_gender         6241 non-null   object 
 2   u_date_of_birth  6239 non-null   object 
 3   SA1              6021 non-null   float64
 4   SA2              6021 non-null   float64
 5   SA3              6021 non-null   float64
 6   SA4              6021 non-null   float64
dtypes: float64(4), object(3)
memory usage: 341.4+ KB
None
            u_disabilities u_gender u_date_of_birth        SA1      SA2  \
0                                        1922-11-30  3100614.0  31006.0   
1                                        2007-04-06  1144614.0  11446.0   
2                            Female      1954-03-26        0.0      0.0   
3                            Female      1978-12-24  3138504.0  31385.0   
4                                        1995-

In [90]:
###Invoice claim details
##  add the desired rows in the query from the three tables
df_invoice_claim_details = pd.read_sql("SELECT c.invoice_id, c.state, c.risk_score, c.risk_level, c.risk_warnings, c.unit_price, c.start_date, c.end_date, c.invoiced_units, c.invoiced_unit_price, c.claimed_units, c.claimed_unit_price, c.claimed_date, c.funded_amount, c.funded_date, c.submitted_date, c.approved_date, c.received_date, c.processed_status, c.reconcile_status, c.ndis_cancelled_date, c.cancel_offset_z4_no, i.state, i.is_pre_approved, i.is_resolved, i.is_risk_identified, i.risk_score, i.invoice_total, i.funded_total, i.funded_date, i.claimed_date, i.reviewed_date, i.invoice_date, i.approved_date, i.submitted_date, i.received_date, m.membership_number, m.status, m.risk_factor, m.price_zone_code FROM HH_claim c left join HH_invoice i on c.invoice_id = i.id left join HH_member m on i.member_id = m.id ;", con=conn)
print(df_invoice_claim_details.info())
print(df_invoice_claim_details)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   invoice_id           968874 non-null  int64         
 1   state                968874 non-null  object        
 2   risk_score           968874 non-null  float64       
 3   risk_level           968874 non-null  object        
 4   risk_warnings        41285 non-null   object        
 5   unit_price           968874 non-null  float64       
 6   start_date           968874 non-null  object        
 7   end_date             968874 non-null  object        
 8   invoiced_units       966054 non-null  float64       
 9   invoiced_unit_price  968874 non-null  float64       
 10  claimed_units        965796 non-null  float64       
 11  claimed_unit_price   968442 non-null  float64       
 12  claimed_date         968328 non-null  object        
 13  funded_amount 

In [91]:
df = df_invoice_claim_details
pd.set_option('display.max_columns', 50)
df.head(100)

,invoice_id,state,risk_score,risk_level,risk_warnings,unit_price,start_date,end_date,invoiced_units,invoiced_unit_price,claimed_units,claimed_unit_price,claimed_date,funded_amount,funded_date,submitted_date,approved_date,received_date,processed_status,reconcile_status,ndis_cancelled_date,cancel_offset_z4_no,state,is_pre_approved,is_resolved,is_risk_identified,risk_score,invoice_total,funded_total,funded_date,claimed_date,reviewed_date,invoice_date,approved_date,submitted_date,received_date,membership_number,status,risk_factor,price_zone_code
0,1,PAID,0.0,NO,None,160.00,2020-04-30,2020-04-30,1.0,160.00,1.0,160.00,2020-04-30,160.00,2020-04-30,2020-04-30,None,2020-04-30 00:00:00,SUCCESSFUL,matched,None,None,ALL_PAID,1,1,0,0.0,160.00,160.0,2020-04-30,2020-04-30,None,2020-04-30,2020-04-30 02:56:15,2020-04-30 02:56:15,2020-04-30 00:00:00,277486509,managed,0.0,ACT_NSW_QLD_VIC
1,2,PAID,0.0,NO,None,160.00,2020-05-01,2020-05-01,1.0,160.00,1.0,160.00,2020-05-04,160.00,2020-05-05,2020-05-04,None,2020-05-04 00:00:00,SUCCESSFUL,matched,None,None,ALL_PAID,1,1,0,0.0,160.00,160.0,2020-05-05,2020-05-04,None,2020-05-01,2020-05-04 01:04:42,2020-05-04 01:04:42,2020-05-01 00:00:00,277486509,managed,0.0,ACT_NSW_QLD_VIC
2,3,PAID,0.0,NO,None,80.00,2020-04-23,2020-04-23,1.0,80.00,1.0,80.00,2020-05-07,80.00,2020-05-08,2020-05-07,2020-05-07,2020-05-07 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-07,None,2020-04-23,2020-05-07 04:36:02,2020-05-07 02:47:50,2020-05-06 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
3,4,PAID,0.0,NO,None,80.00,2020-04-30,2020-04-30,1.0,80.00,1.0,80.00,2020-05-07,80.00,2020-05-08,2020-05-07,2020-05-07,2020-05-07 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-07,None,2020-04-30,2020-05-07 04:36:13,2020-05-07 02:48:45,2020-05-06 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
4,5,PAID,0.0,NO,None,80.00,2020-05-07,2020-05-07,1.0,80.00,1.0,80.00,2020-05-08,80.00,2020-05-08,2020-05-08,2020-05-08,2020-05-08 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-08,None,2020-05-07,2020-05-08 01:23:24,2020-05-08 00:49:39,2020-05-07 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,88,PAID,0.0,NO,None,41.04,2019-10-21,2019-10-21,4.0,41.04,4.0,41.04,2019-12-20,164.16,2019-12-20,2019-12-20,None,2019-12-18 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,389.88,NaN,None,2019-12-20,None,2019-11-05,2019-12-20 06:42:31,2019-12-20 00:29:20,2019-12-18 00:00:00,484394812,managed,0.0,ACT_NSW_QLD_VIC
96,65,PAID,0.0,NO,None,13.82,2019-12-04,2019-12-04,25.0,13.82,25.0,13.82,2019-12-04,345.50,2019-12-04,2019-12-04,None,2019-12-01 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,345.50,NaN,None,2019-12-04,None,2019-12-04,2019-12-04 03:42:41,2019-12-04 03:29:14,2019-12-01 00:00:00,484394812,managed,0.0,ACT_NSW_QLD_VIC
97,84,PAID,0.0,NO,None,11.27,2020-05-07,2020-05-07,10.0,11.27,10.0,11.27,2020-05-11,112.70,2020-05-11,2020-05-11,None,2020-05-11 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,112.70,NaN,None,2020-05-11,None,2020-05-07,2020-05-11 07:18:45,2020-05-11 06:11:17,2020-05-11 00:00:00,484394812,managed,0.0,ACT_NSW_QLD_VIC
98,66,PAID,0.0,NO,None,47.46,2020-01-24,2020-01-24,5.5,47.46,5.5,47.46,2020-03-20,261.03,2020-03-20,2020-03-20,None,2020-03-18 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,319.84,NaN,None,2020-03-20,None,2020-01-29,2020-03-20 03:00:59,2020-03-20 02:49:29,2020-03-18 00:00:00,484394812,managed,0.0,ACT_NSW_QLD_VIC


In [92]:
df.tail(10)


,invoice_id,state,risk_score,risk_level,risk_warnings,unit_price,start_date,end_date,invoiced_units,invoiced_unit_price,claimed_units,claimed_unit_price,claimed_date,funded_amount,funded_date,submitted_date,approved_date,received_date,processed_status,reconcile_status,ndis_cancelled_date,cancel_offset_z4_no,state,is_pre_approved,is_resolved,is_risk_identified,risk_score,invoice_total,funded_total,funded_date,claimed_date,reviewed_date,invoice_date,approved_date,submitted_date,received_date,membership_number,status,risk_factor,price_zone_code
968864,539311,OPERATOR_AUTHORIZED,0.0,NO,None,0.85,2020-12-03,2020-12-31,217.0,0.85,217.0,0.85,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,184.45,NaN,None,None,None,2021-01-28,2021-04-19 09:21:59,2021-04-19 09:21:59,2021-04-15 00:00:00,663618020,managed,0.0,ACT_NSW_QLD_VIC
968865,539312,OPERATOR_AUTHORIZED,0.0,NO,None,0.85,2021-02-01,2021-02-25,140.0,0.85,140.0,0.85,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,119.00,NaN,None,None,None,2021-03-18,2021-04-19 09:22:30,2021-04-19 09:22:30,2021-04-15 00:00:00,663618020,managed,0.0,ACT_NSW_QLD_VIC
968866,539313,OPERATOR_AUTHORIZED,0.0,NO,None,50.20,2021-04-12,2021-04-12,1.0,50.20,1.0,50.20,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,50.20,NaN,None,None,None,2021-04-15,2021-04-19 09:23:40,2021-04-19 09:23:40,2021-04-15 00:00:00,972904711,managed,0.0,ACT_NSW_QLD_VIC
968867,539314,OPERATOR_AUTHORIZED,0.0,NO,None,49.30,2021-04-09,2021-04-09,1.0,49.30,1.0,49.30,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,49.30,NaN,None,None,None,2021-04-15,2021-04-19 09:24:25,2021-04-19 09:24:25,2021-04-15 00:00:00,396451535,managed,0.0,ACT_NSW_QLD_VIC
968868,539315,OPERATOR_AUTHORIZED,0.0,NO,None,122.51,2021-04-05,2021-04-05,6.0,122.51,6.0,122.51,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,2135.70,NaN,None,None,None,2021-04-15,2021-04-19 09:25:29,2021-04-19 09:25:29,2021-04-15 00:00:00,265655619,managed,0.0,ACT_NSW_QLD_VIC
968869,539315,OPERATOR_AUTHORIZED,0.0,NO,None,55.47,2021-04-08,2021-04-08,6.0,55.47,6.0,55.47,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,2135.70,NaN,None,None,None,2021-04-15,2021-04-19 09:25:29,2021-04-19 09:25:29,2021-04-15 00:00:00,265655619,managed,0.0,ACT_NSW_QLD_VIC
968870,539315,OPERATOR_AUTHORIZED,0.0,NO,None,77.81,2021-04-10,2021-04-10,6.0,77.81,6.0,77.81,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,2135.70,NaN,None,None,None,2021-04-15,2021-04-19 09:25:29,2021-04-19 09:25:29,2021-04-15 00:00:00,265655619,managed,0.0,ACT_NSW_QLD_VIC
968871,539315,OPERATOR_AUTHORIZED,0.0,NO,None,100.16,2021-04-11,2021-04-11,6.0,100.16,6.0,100.16,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,2135.70,NaN,None,None,None,2021-04-15,2021-04-19 09:25:29,2021-04-19 09:25:29,2021-04-15 00:00:00,265655619,managed,0.0,ACT_NSW_QLD_VIC
968872,539316,OPERATOR_AUTHORIZED,0.0,NO,None,55.47,2021-04-15,2021-04-15,6.0,55.47,6.0,55.47,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,396.57,NaN,None,None,None,2021-04-15,2021-04-19 09:26:45,2021-04-19 09:26:45,2021-04-15 00:00:00,615053396,managed,0.0,ACT_NSW_QLD_VIC
968873,539316,OPERATOR_AUTHORIZED,0.0,NO,None,0.85,2021-04-15,2021-04-15,75.0,0.85,75.0,0.85,2021-04-19,NaN,None,2021-04-19,None,2021-04-19 00:00:00,None,None,None,None,PENDING_CLAIM,1,1,0,0.0,396.57,NaN,None,None,None,2021-04-15,2021-04-19 09:26:45,2021-04-19 09:26:45,2021-04-15 00:00:00,615053396,managed,0.0,ACT_NSW_QLD_VIC


In [93]:
print(df.columns.tolist())

['invoice_id', 'state', 'risk_score', 'risk_level', 'risk_warnings', 'unit_price', 'start_date', 'end_date', 'invoiced_units', 'invoiced_unit_price', 'claimed_units', 'claimed_unit_price', 'claimed_date', 'funded_amount', 'funded_date', 'submitted_date', 'approved_date', 'received_date', 'processed_status', 'reconcile_status', 'ndis_cancelled_date', 'cancel_offset_z4_no', 'state', 'is_pre_approved', 'is_resolved', 'is_risk_identified', 'risk_score', 'invoice_total', 'funded_total', 'funded_date', 'claimed_date', 'reviewed_date', 'invoice_date', 'approved_date', 'submitted_date', 'received_date', 'membership_number', 'status', 'risk_factor', 'price_zone_code']


In [94]:
df['state'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   state   968874 non-null  object
 1   state   968874 non-null  object
dtypes: object(2)
memory usage: 14.8+ MB


In [95]:
df['state'].head(100)


,state,state
0,PAID,ALL_PAID
1,PAID,ALL_PAID
2,PAID,ALL_PAID
3,PAID,ALL_PAID
4,PAID,ALL_PAID
...,...,...
95,PAID,ALL_PAID
96,PAID,ALL_PAID
97,PAID,ALL_PAID
98,PAID,ALL_PAID


In [96]:
df['state'].tail(100)

,state,state
968774,OPERATOR_AUTHORIZED,PENDING_CLAIM
968775,OPERATOR_AUTHORIZED,PENDING_CLAIM
968776,OPERATOR_AUTHORIZED,PENDING_CLAIM
968777,OPERATOR_AUTHORIZED,PENDING_CLAIM
968778,OPERATOR_AUTHORIZED,PENDING_CLAIM
...,...,...
968869,OPERATOR_AUTHORIZED,PENDING_CLAIM
968870,OPERATOR_AUTHORIZED,PENDING_CLAIM
968871,OPERATOR_AUTHORIZED,PENDING_CLAIM
968872,OPERATOR_AUTHORIZED,PENDING_CLAIM


### There are two state columns which stand for different state of claim, let's rename the second one as final_status - final_status is an important variable 

In [97]:
df['risk_score'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   risk_score  968874 non-null  float64
 1   risk_score  968874 non-null  float64
dtypes: float64(2)
memory usage: 14.8 MB


In [98]:
df['risk_score'].describe()

,risk_score,risk_score
count,968874.000000,968874.000000
mean,0.029622,0.029622
std,1.720851,1.720851
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,100.000000,100.000000


###  => Drop 2 risk_score columns 

In [99]:
#df['risk_level'].tail(100)
df['risk_level'].head(100)


0     NO
1     NO
2     NO
3     NO
4     NO
      ..
95    NO
96    NO
97    NO
98    NO
99    NO
Name: risk_level, Length: 100, dtype: object

### => Keep column risk_level

In [100]:
df['risk_warnings'].value_counts()

Claimable total has changed                40998
Score: 100, Member flagged as high risk      287
Name: risk_warnings, dtype: int64

### risk_warnings has the same values with risk_level => drop this column

In [101]:
# We can see the invoiced_unit_price and unit_price probably have the same values, let's check:
df['unit_price'].equals(df['invoiced_unit_price'])


True

### Drop unit_price either invoice_unit_price or both of them


In [102]:
df['invoiced_units'].value_counts()

1.00       262875
2.00        92638
3.00        59192
4.00        50242
6.00        35023
            ...  
877.11          1
1256.00         1
1087.00         1
1489.51         1
17.48           1
Name: invoiced_units, Length: 14709, dtype: int64

In [103]:
df['invoiced_units'].describe()

count    966054.000000
mean         17.431631
std         181.127771
min         -43.000000
25%           1.000000
50%           2.000000
75%           5.230000
max       45000.000000
Name: invoiced_units, dtype: float64

In [111]:
df['invoiced_units'].tail(10)

968864    217.0
968865    140.0
968866      1.0
968867      1.0
968868      6.0
968869      6.0
968870      6.0
968871      6.0
968872      6.0
968873     75.0
Name: invoiced_units, dtype: float64

### Drop this column as the invoice_total indicates the final number

In [113]:
df['price_zone_code'].value_counts()

ACT_NSW_QLD_VIC    917679
NT_SA_TAS_WA        40993
Remote               8490
VeryRemote           1712
Name: price_zone_code, dtype: int64

In [114]:
df['funded_total'].value_counts()

193.99     15330
214.41      7031
0.00        6242
290.99      4586
97.00       3548
           ...  
79.50          1
107.75         1
5433.61        1
677.14         1
677.66         1
Name: funded_total, Length: 52528, dtype: int64

### Create final_df to work on by grouping the interested_columns

In [105]:
print(df.columns.tolist())

['invoice_id', 'state', 'risk_score', 'risk_level', 'risk_warnings', 'unit_price', 'start_date', 'end_date', 'invoiced_units', 'invoiced_unit_price', 'claimed_units', 'claimed_unit_price', 'claimed_date', 'funded_amount', 'funded_date', 'submitted_date', 'approved_date', 'received_date', 'processed_status', 'reconcile_status', 'ndis_cancelled_date', 'cancel_offset_z4_no', 'state', 'is_pre_approved', 'is_resolved', 'is_risk_identified', 'risk_score', 'invoice_total', 'funded_total', 'funded_date', 'claimed_date', 'reviewed_date', 'invoice_date', 'approved_date', 'submitted_date', 'received_date', 'membership_number', 'status', 'risk_factor', 'price_zone_code']


In [106]:
df.head(200)

,invoice_id,state,risk_score,risk_level,risk_warnings,unit_price,start_date,end_date,invoiced_units,invoiced_unit_price,claimed_units,claimed_unit_price,claimed_date,funded_amount,funded_date,submitted_date,approved_date,received_date,processed_status,reconcile_status,ndis_cancelled_date,cancel_offset_z4_no,state,is_pre_approved,is_resolved,is_risk_identified,risk_score,invoice_total,funded_total,funded_date,claimed_date,reviewed_date,invoice_date,approved_date,submitted_date,received_date,membership_number,status,risk_factor,price_zone_code
0,1,PAID,0.0,NO,None,160.00,2020-04-30,2020-04-30,1.0,160.00,1.0,160.00,2020-04-30,160.00,2020-04-30,2020-04-30,None,2020-04-30 00:00:00,SUCCESSFUL,matched,None,None,ALL_PAID,1,1,0,0.0,160.00,160.0,2020-04-30,2020-04-30,None,2020-04-30,2020-04-30 02:56:15,2020-04-30 02:56:15,2020-04-30 00:00:00,277486509,managed,0.0,ACT_NSW_QLD_VIC
1,2,PAID,0.0,NO,None,160.00,2020-05-01,2020-05-01,1.0,160.00,1.0,160.00,2020-05-04,160.00,2020-05-05,2020-05-04,None,2020-05-04 00:00:00,SUCCESSFUL,matched,None,None,ALL_PAID,1,1,0,0.0,160.00,160.0,2020-05-05,2020-05-04,None,2020-05-01,2020-05-04 01:04:42,2020-05-04 01:04:42,2020-05-01 00:00:00,277486509,managed,0.0,ACT_NSW_QLD_VIC
2,3,PAID,0.0,NO,None,80.00,2020-04-23,2020-04-23,1.0,80.00,1.0,80.00,2020-05-07,80.00,2020-05-08,2020-05-07,2020-05-07,2020-05-07 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-07,None,2020-04-23,2020-05-07 04:36:02,2020-05-07 02:47:50,2020-05-06 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
3,4,PAID,0.0,NO,None,80.00,2020-04-30,2020-04-30,1.0,80.00,1.0,80.00,2020-05-07,80.00,2020-05-08,2020-05-07,2020-05-07,2020-05-07 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-07,None,2020-04-30,2020-05-07 04:36:13,2020-05-07 02:48:45,2020-05-06 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
4,5,PAID,0.0,NO,None,80.00,2020-05-07,2020-05-07,1.0,80.00,1.0,80.00,2020-05-08,80.00,2020-05-08,2020-05-08,2020-05-08,2020-05-08 00:00:00,SUCCESSFUL,TO_BE_RECONCILED,None,None,ALL_PAID,0,1,0,0.0,80.00,80.0,2020-05-08,2020-05-08,None,2020-05-07,2020-05-08 01:23:24,2020-05-08 00:49:39,2020-05-07 00:00:00,815939440,managed,0.0,ACT_NSW_QLD_VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,396,PAID,0.0,NO,None,41.04,2019-10-21,2019-10-21,5.0,41.04,5.0,41.04,2019-11-06,205.20,2019-11-06,2019-11-06,None,2019-11-05 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,205.20,NaN,None,2019-11-06,None,2019-11-05,2019-11-06 02:42:52,2019-11-06 00:14:21,2019-11-05 00:00:00,832922816,managed,0.0,ACT_NSW_QLD_VIC
196,282,PAID,0.0,NO,None,55.66,2020-04-17,2020-04-17,1.0,55.66,1.0,55.66,2020-04-21,55.66,2020-04-21,2020-04-20,None,2020-04-17 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,55.66,NaN,None,2020-04-21,None,2020-04-17,2020-04-21 01:44:38,2020-04-20 01:29:32,2020-04-17 00:00:00,832922816,managed,0.0,ACT_NSW_QLD_VIC
197,221,PAID,0.0,NO,None,235.85,2020-05-27,2020-05-27,2.0,235.85,2.0,235.85,2020-06-06,471.70,2020-06-06,2020-06-03,None,2020-06-01 00:00:00,None,None,None,None,ALL_PAID,0,1,0,0.0,825.48,NaN,None,2020-06-06,None,2020-06-01,2020-06-06 09:47:38,2020-06-03 00:24:26,2020-06-01 00:00:00,832922816,managed,0.0,ACT_NSW_QLD_VIC
198,235,PAID,0.0,NO,None,179.26,2019-04-15,2019-04-15,0.0,179.26,0.0,179.26,2019-05-08,537.78,2019-05-08,2019-05-08,None,None,None,None,None,None,ALL_PAID,0,1,0,0.0,537.78,NaN,None,2019-05-08,None,2019-04-30,2019-05-08 02:52:22,2019-05-08 02:52:21,None,984761030,managed,0.0,ACT_NSW_QLD_VIC


In [107]:
# Explore the same technique we update the drop list:
drop_columns = ['risk_score','risk_warnings','unit_price','invoiced_units','invoiced_unit_price','claimed_units','claimed_unit_price','is_pre_approved','is_resolved', 'is_risk_identified','approved_date', 'received_date', 'reviewed_date', 'invoice_date', 'approved_date','submitted_date','received_date','risk_factor']

In [120]:
interested_columns = ['invoice_id', 'state', 'risk_level', 'start_date', 'invoice_total', 'funded_total', 'funded_date','membership_number', 'price_zone_code']

In [126]:
final_df = df[interested_columns]

In [127]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   invoice_id         968874 non-null  int64  
 1   state              968874 non-null  object 
 2   state              968874 non-null  object 
 3   risk_level         968874 non-null  object 
 4   start_date         968874 non-null  object 
 5   invoice_total      968874 non-null  float64
 6   funded_total       566458 non-null  float64
 7   funded_date        958743 non-null  object 
 8   funded_date        556790 non-null  object 
 9   membership_number  968874 non-null  object 
 10  price_zone_code    968874 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 81.3+ MB


In [130]:
#drop funded_date with lower input 
final_df = final_df.drop(final_df.columns[8],axis=1)

In [132]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   invoice_id         968874 non-null  int64  
 1   state              968874 non-null  object 
 2   state              968874 non-null  object 
 3   risk_level         968874 non-null  object 
 4   start_date         968874 non-null  object 
 5   invoice_total      968874 non-null  float64
 6   funded_total       566458 non-null  float64
 7   membership_number  968874 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 59.1+ MB


### => Need to pick more table, firstly to get a table with invoice_id and service its paid for, secondly we need to add membership_number to user table then we can merge later on